In [1]:
import sys
sys.path.append('SimuQ')
%load_ext autoreload
%autoreload 2
import numpy as np
from simuq.qsystem import QSystem
from simuq.environment import qubit
from scipy.constants import h

chain=False
n = 6
T = 1

qs = QSystem()
qubits = [qubit(qs) for i in range(n)]
H0 = 0
# H1 = 0
if chain:
    for i in range(n-1):
        H0 += qubits[i].Z * qubits[(i+1)%n].Z
else:
    for i in range(n):
        H0 += qubits[i].Z * qubits[(i+1)%n].Z

for i in range(n):
    H0 += - qubits[i].X

qs.add_evolution(H0, T)
from simuq.solver import generate_as
# from aais.rydberg1d_aws import GenMach
# from backends.bloqade_rydberg_aws import transpile
from aais.heis_aais import GenMach
mach=GenMach(n)
from qiskit_ionq import IonQProvider
provider = IonQProvider("HrGkI4NCFf3uuM8sooNAQXePJfFTvWfd")

In [54]:
from backends.ionq_qiskit_qis import transpile
simulator_backend = provider.get_backend("ionq_simulator",gateset="qis")
circ = transpile(*generate_as(qs, mach,10))
# circ.draw()
job = simulator_backend.run(circ, shots=10000) 
print(job.get_counts())

First round time:  0.06867408752441406
Second round time:  0.014994382858276367
{'000000': 2995, '000001': 227, '000010': 189, '000011': 297, '000100': 226, '000101': 19, '000110': 260, '000111': 261, '001000': 224, '001001': 33, '001010': 11, '001011': 7, '001100': 244, '001101': 8, '001110': 288, '001111': 203, '010000': 223, '010001': 19, '010010': 26, '010011': 4, '010100': 15, '010101': 3, '010110': 4, '010111': 1, '011000': 282, '011001': 3, '011010': 5, '011011': 30, '011100': 249, '011101': 0, '011110': 208, '011111': 74, '100000': 209, '100001': 310, '100010': 12, '100011': 271, '100100': 33, '100101': 5, '100110': 3, '100111': 226, '101000': 18, '101001': 4, '101010': 3, '101011': 0, '101100': 6, '101101': 37, '101110': 0, '101111': 73, '110000': 268, '110001': 249, '110010': 4, '110011': 232, '110100': 6, '110101': 0, '110110': 40, '110111': 74, '111000': 285, '111001': 226, '111010': 0, '111011': 71, '111100': 257, '111101': 92, '111110': 72, '111111': 276}


In [55]:
from backends.ionq_qiskit_native import transpile,debug_circ

simulator_backend = provider.get_backend("ionq_simulator",gateset="native")
circ = transpile(*generate_as(qs, mach,10))
job = simulator_backend.run(circ, shots=10000) 
print(job.get_counts())

First round time:  0.06866979598999023
Second round time:  0.01130056381225586


{'000000': 2860, '000001': 226, '000010': 253, '000011': 256, '000100': 221, '000101': 15, '000110': 256, '000111': 302, '001000': 206, '001001': 33, '001010': 17, '001011': 7, '001100': 262, '001101': 5, '001110': 274, '001111': 226, '010000': 209, '010001': 16, '010010': 29, '010011': 12, '010100': 10, '010101': 5, '010110': 2, '010111': 0, '011000': 294, '011001': 4, '011010': 8, '011011': 26, '011100': 250, '011101': 0, '011110': 249, '011111': 85, '100000': 244, '100001': 255, '100010': 20, '100011': 262, '100100': 27, '100101': 7, '100110': 5, '100111': 253, '101000': 14, '101001': 5, '101010': 0, '101011': 1, '101100': 6, '101101': 36, '101110': 0, '101111': 89, '110000': 280, '110001': 276, '110010': 3, '110011': 223, '110100': 3, '110101': 0, '110110': 37, '110111': 83, '111000': 296, '111001': 230, '111010': 0, '111011': 79, '111100': 224, '111101': 83, '111110': 89, '111111': 252}


In [52]:
from backends.ionq_qiskit_native import transpile,debug_circ

simulator_backend = provider.get_backend("ionq_simulator",gateset="native")
circ = debug_circ()
job = simulator_backend.run(circ, shots=10000) 
print(job.get_counts())

{'00': 4972, '01': 5028}


In [2]:
from backends.ionq import transpile
import json

for chain in [True,False]:
    for T in [0.2,0.4,0.6,0.8,1]:
        n = 6

        qs = QSystem()
        qubits = [qubit(qs) for i in range(n)]
        H0 = 0
        # H1 = 0
        if chain:
            for i in range(n-1):
                H0 += qubits[i].Z * qubits[(i+1)%n].Z
        else:
            for i in range(n):
                H0 += qubits[i].Z * qubits[(i+1)%n].Z

        for i in range(n):
            H0 += - qubits[i].X

        qs.add_evolution(H0, T)
        # circ = debug_circ().job
        circ=transpile(*generate_as(qs, mach,14),backend="simulator",noise_model="aria-1")
        if not chain:
            file_name=f'{T}_cycle.json'
        else:
            file_name=f'{T}_chain.json'
        with open(file_name, 'w') as f:
            json.dump(circ, f)
    

First round time:  0.061197757720947266
Second round time:  0.010908126831054688
First round time:  0.060761213302612305
Second round time:  0.01007223129272461
First round time:  0.06066775321960449
Second round time:  0.010578393936157227
First round time:  0.06153082847595215
Second round time:  0.010087251663208008
First round time:  0.061293840408325195
Second round time:  0.010667085647583008
First round time:  0.06839585304260254
Second round time:  0.011442422866821289
First round time:  0.06842470169067383
Second round time:  0.01134634017944336
First round time:  0.06811213493347168
Second round time:  0.011169672012329102
First round time:  0.07160329818725586
Second round time:  0.012233972549438477
First round time:  0.0689084529876709
Second round time:  0.011165380477905273


In [72]:
import os
import subprocess
bashCommand = """curl -X POST "https://api.ionq.co/v0.3/jobs" \
-H "Authorization: apiKey HrGkI4NCFf3uuM8sooNAQXePJfFTvWfd" \
-H "Content-Type: application/json" \
-d @1_cycle.json"""
output = subprocess.getoutput(bashCommand)
id=eval("{"+output.split("{")[1])["id"]
id

'32b40a97-47af-49f2-8ded-6a2fac44054f'

In [67]:
bashCommand = f"curl \"https://api.ionq.co/v0.1/jobs/"+str(id)+"\" \
-H \"Authorization: apiKey HrGkI4NCFf3uuM8sooNAQXePJfFTvWfd\" "
output = subprocess.getoutput(bashCommand)
output.split("\n")[-1]


'{"noise":{"model":"aria-1","seed":524331435},"status":"completed","name":"test","target":"simulator","shots":4096,"predicted_execution_time":64,"execution_time":1496,"id":"c061fbfc-0439-4c7d-b5ff-7e467f5854ba","qubits":6,"type":"circuit","request":1688023579,"start":1688023582,"response":1688023583,"gate_counts":{"1q":630,"2q":42},"data":{"histogram":{"0": 0.1823730469, "1": 0.033203125, "2": 0.0341796875, "3": 0.02905273438, "4": 0.0341796875, "5": 0.004150390625, "6": 0.02905273438, "7": 0.02587890625, "8": 0.02783203125, "9": 0.008056640625, "10": 0.004638671875, "11": 0.00244140625, "12": 0.02612304688, "13": 0.0068359375, "14": 0.02221679688, "15": 0.02075195312, "16": 0.03466796875, "17": 0.007080078125, "18": 0.0029296875, "19": 0.00537109375, "20": 0.00537109375, "21": 0.001953125, "22": 0.006103515625, "23": 0.003173828125, "24": 0.02587890625, "25": 0.006103515625, "26": 0.004150390625, "27": 0.007080078125, "28": 0.02319335938, "29": 0.003662109375, "30": 0.01635742188, "31

In [19]:
a="""{"id":"8a9ed38a-df1a-4e66-80db-2a54a7d8acfe","status":"ready","request":1688021253}{"id":"00b06274-97a6-47b5-b0bd-e56e343522fe","status":"ready","request":1688021253}{"id":"0b3a2120-19f1-46a0-9059-eb0480662c65","status":"ready","request":1688021254}{"id":"e8270631-ecc9-4fbc-a91e-96b996f2334f","status":"ready","request":1688021254}{"id":"95fe1239-0fc1-4152-bb83-95f0685d7449","status":"ready","request":1688021254}{"id":"9c15dea9-ca70-4487-965d-08b936bb5c66","status":"ready","request":1688021254}{"id":"cb8af364-c748-451c-9346-c14851043e04","status":"ready","request":1688021255}{"id":"cdd45ce1-a955-46f6-9c6f-25b3410c77e2","status":"ready","request":1688021255}{"id":"14a408ca-35a2-4698-a986-9056aad1d8a3","status":"ready","request":1688021255}{"id":"bfff6ebe-d566-4bc3-a53d-d6dbc2ef000a","status":"ready","request":1688021255}"""

In [20]:
for i in range(10):
    if i==0:
        print("chain")
    if i==5:
        print("cycle")
    b=a.split("}")
    print(f"{((i)%5+1)/5}:"+eval(b[i]+"}")["id"])


chain
0.2:8a9ed38a-df1a-4e66-80db-2a54a7d8acfe
0.4:00b06274-97a6-47b5-b0bd-e56e343522fe
0.6:0b3a2120-19f1-46a0-9059-eb0480662c65
0.8:e8270631-ecc9-4fbc-a91e-96b996f2334f
1.0:95fe1239-0fc1-4152-bb83-95f0685d7449
cycle
0.2:9c15dea9-ca70-4487-965d-08b936bb5c66
0.4:cb8af364-c748-451c-9346-c14851043e04
0.6:cdd45ce1-a955-46f6-9c6f-25b3410c77e2
0.8:14a408ca-35a2-4698-a986-9056aad1d8a3
1.0:bfff6ebe-d566-4bc3-a53d-d6dbc2ef000a
